## Binary Classification (Logistic Regression) - Breast Cancer [pytorch]

### Data

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_test_scaled.shape, y_test.shape)

(455, 30) (455,)
(114, 30) (114,)


### [scikit-learn] Modeling and Training

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# sklearn model
model = LogisticRegression()
model.fit(x_train_scaled, y_train)

# Training
y_pred = model.predict(x_train_scaled)
acc = accuracy_score(y_train, y_pred)
print(f'Train Accuracy: {acc:.4f}')

# Evaluation
y_pred = model.predict(x_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f'Test  Accuracy: {acc:.4f}')

Train Accuracy: 0.9868
Test  Accuracy: 0.9737


### [pytorch-1] Modeling and Training

- Manual backward propagation
- Manual update of weights and biases

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

def bce_loss(y_pred, y_true):
    eps = 1e-8
    return -torch.mean(y_true*torch.log(y_pred + eps) + (1 - y_true)*torch.log(1 - y_pred + eps))

def binary_accuracy(x_pred, x_true):
    return torch.eq(x_pred.round(), x_true.round()).float().mean() 

## Data
x = torch.tensor(x_train_scaled).float()
y = torch.tensor(y_train).float().view(-1, 1)
print(x.shape, y.shape)

torch.Size([455, 30]) torch.Size([455, 1])


In [10]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 1

w1 = torch.randn(input_size, hidden_size)
b1 = torch.zeros(hidden_size)
w2 = torch.randn(hidden_size, output_size)
b2 = torch.zeros(output_size)

## Train
n_epochs = 10000
learning_rate = 0.01

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = torch.sigmoid(z2)

    loss = bce_loss(out, y)
    score = binary_accuracy(out, y)

    # # Backward progapation
    grad_out = (out - y) / out / (1 - out) / y.shape[0]
    grad_z2 = out * (1 - out) * grad_out
    grad_w2 = torch.mm(a1.T, grad_z2)
    grad_b2 = torch.sum(grad_z2, dim=0)

    grad_a1 = torch.mm(grad_z2, w2.T)
    grad_z1 = a1 * (1 - a1) * grad_a1
    grad_w1 = torch.mm(x.T, grad_z1)
    grad_b1 = torch.sum(grad_z1, dim=0)

    # # Update weights and biases
    w1 -= learning_rate * grad_w1
    b1 -= learning_rate * grad_b1
    w2 -= learning_rate * grad_w2
    b2 -= learning_rate * grad_b2

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 0.13 score: 0.9451
[2000/10000] loss: 0.09 score: 0.9670
[3000/10000] loss: 0.08 score: 0.9780
[4000/10000] loss: 0.06 score: 0.9846
[5000/10000] loss: 0.06 score: 0.9846
[6000/10000] loss: 0.05 score: 0.9846
[7000/10000] loss: 0.05 score: 0.9868
[8000/10000] loss: 0.04 score: 0.9912
[9000/10000] loss: 0.04 score: 0.9912
[10000/10000] loss: 0.04 score: 0.9912


### [pytorch-2] Modeling and Training

- Automatic backward propagation: torch.autograd.grad()
- Manual update of weights and biases

In [11]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.01

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = torch.sigmoid(z2)

    loss = bce_loss(out, y)
    score = binary_accuracy(out, y)

    # # Backward progapation
    grads = torch.autograd.grad(loss, [w1, b1, w2, b2], create_graph=True)

    # Update weights and biases
    with torch.no_grad():
        w1 -= learning_rate * grads[0]
        b1 -= learning_rate * grads[1]
        w2 -= learning_rate * grads[2]
        b2 -= learning_rate * grads[3]

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 0.13 score: 0.9451
[2000/10000] loss: 0.09 score: 0.9670
[3000/10000] loss: 0.08 score: 0.9780
[4000/10000] loss: 0.06 score: 0.9846
[5000/10000] loss: 0.06 score: 0.9846
[6000/10000] loss: 0.05 score: 0.9846
[7000/10000] loss: 0.05 score: 0.9868
[8000/10000] loss: 0.04 score: 0.9912
[9000/10000] loss: 0.04 score: 0.9912
[10000/10000] loss: 0.04 score: 0.9912


### [pytorch-3] Modeling and Training

- Automatic backward propagation: loss.backward()
- Manual update of weights and biases

In [12]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.01

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = torch.sigmoid(z2)

    loss = bce_loss(out, y)
    score = binary_accuracy(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        b1 -= learning_rate * b1.grad
        w2 -= learning_rate * w2.grad
        b2 -= learning_rate * b2.grad

        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 0.13 score: 0.9451
[2000/10000] loss: 0.09 score: 0.9670
[3000/10000] loss: 0.08 score: 0.9780
[4000/10000] loss: 0.06 score: 0.9846
[5000/10000] loss: 0.06 score: 0.9846
[6000/10000] loss: 0.05 score: 0.9846
[7000/10000] loss: 0.05 score: 0.9868
[8000/10000] loss: 0.04 score: 0.9912
[9000/10000] loss: 0.04 score: 0.9912
[10000/10000] loss: 0.04 score: 0.9912


### [pytorch-4] Modeling and Training

- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [13]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.01

optimizer = optim.SGD([w1, b1, w2, b2], lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = torch.sigmoid(z2)

    loss = bce_loss(out, y)
    score = binary_accuracy(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 0.13 score: 0.9451
[2000/10000] loss: 0.09 score: 0.9670
[3000/10000] loss: 0.08 score: 0.9780
[4000/10000] loss: 0.06 score: 0.9846
[5000/10000] loss: 0.06 score: 0.9846
[6000/10000] loss: 0.05 score: 0.9846
[7000/10000] loss: 0.05 score: 0.9868
[8000/10000] loss: 0.04 score: 0.9912
[9000/10000] loss: 0.04 score: 0.9912
[10000/10000] loss: 0.04 score: 0.9912


### [pytorch-5] Modeling and Training

- MLP model: torch.nn.Linear()
- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [ ]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 1

model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.Sigmoid(),
    nn.Linear(hidden_size, output_size),
    nn.Sigmoid(),
)

## Train
n_epochs = 10000
learning_rate = 0.01

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    out = model(x)

    loss = bce_loss(out, y)
    score = binary_accuracy(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 0.18 score: 0.9495
[2000/10000] loss: 0.12 score: 0.9714
[3000/10000] loss: 0.10 score: 0.9758
[4000/10000] loss: 0.09 score: 0.9824
[5000/10000] loss: 0.08 score: 0.9824
[6000/10000] loss: 0.07 score: 0.9824
[7000/10000] loss: 0.07 score: 0.9846
[8000/10000] loss: 0.07 score: 0.9846
[9000/10000] loss: 0.07 score: 0.9846
[10000/10000] loss: 0.06 score: 0.9846


### [pytorch-6] Modeling and Training

- MLP model: torch.nn.Linear() with initialization
- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [ ]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 1

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
        ## initialization
        torch.nn.init.normal_(self.linear1.weight)
        torch.nn.init.normal_(self.linear2.weight)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)  
        x = torch.sigmoid(x)
        return x

model = MLP(input_size, hidden_size, output_size)

## Train
n_epochs = 10000
learning_rate = 0.01

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    out = model(x)

    loss = loss_fn(out, y)
    score = binary_accuracy(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 0.17 score: 0.9363
[2000/10000] loss: 0.11 score: 0.9626
[3000/10000] loss: 0.09 score: 0.9736
[4000/10000] loss: 0.07 score: 0.9780
[5000/10000] loss: 0.06 score: 0.9780
[6000/10000] loss: 0.06 score: 0.9846
[7000/10000] loss: 0.05 score: 0.9890
[8000/10000] loss: 0.05 score: 0.9912
[9000/10000] loss: 0.04 score: 0.9912
[10000/10000] loss: 0.04 score: 0.9912


### [pytorch-7] Modeling and Training (Multiclass Classification)

- Model output: logit with without actiavtion function
- Loss function: nn.CrossEntropyLoss() (No one-hot encoding, long type)
- MLP model: torch.nn.Linear() with initialization
- Automatic backward propagation: loss.backward()
- Automatic update of weights and biases: optimizer

In [16]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 30, 100, 2

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
        ## initialization
        torch.nn.init.normal_(self.linear1.weight)
        torch.nn.init.normal_(self.linear2.weight)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)  
        return x            # No activation for logit

def accuracy(y_pred, y_true):
    y_pred = y_pred.argmax(dim=1)
    return torch.eq(y_pred, y_true).float().mean()

model = MLP(input_size, hidden_size, output_size)

## Train
n_epochs = 1000
learning_rate = 0.01

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    out = model(x)

    loss = loss_fn(out, y.squeeze().long())
    score = accuracy(out, y.squeeze().long())

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    optimizer.step()
    optimizer.zero_grad()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[100/1000] loss: 0.59 score: 0.8374
[200/1000] loss: 0.37 score: 0.8813
[300/1000] loss: 0.27 score: 0.9187
[400/1000] loss: 0.21 score: 0.9253
[500/1000] loss: 0.17 score: 0.9363
[600/1000] loss: 0.14 score: 0.9429
[700/1000] loss: 0.12 score: 0.9516
[800/1000] loss: 0.11 score: 0.9582
[900/1000] loss: 0.11 score: 0.9582
[1000/1000] loss: 0.10 score: 0.9626
